In [2]:
import os
import re
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from vowpalwabbit import pyvw

import nltk
nltk.download('stopwords')

In [2]:
path = "data/aclImdb/"

positiveFiles = [x for x in os.listdir(path+"train/pos/") if x.endswith(".txt")]
negativeFiles = [x for x in os.listdir(path+"train/neg/") if x.endswith(".txt")]
testPosFiles = [x for x in os.listdir(path+"test/pos/") if x.endswith(".txt")]
testNegFiles = [x for x in os.listdir(path+"test/neg/") if x.endswith(".txt")]

In [3]:
display(len(positiveFiles))
display(len(negativeFiles))
display(len(testPosFiles))
display(len(testNegFiles))

12500

12500

12500

12500

In [4]:
positiveReviews, negativeReviews, testPosReviews, testNegReviews = [], [], [], []
for pfile in positiveFiles:
    with open(path+"train/pos/"+pfile, encoding="latin1") as f:
        positiveReviews.append(f.read())
for nfile in negativeFiles:
    with open(path+"train/neg/"+nfile, encoding="latin1") as f:
        negativeReviews.append(f.read())
for tfile in testPosFiles:
    with open(path+"test/pos/"+tfile, encoding="latin1") as f:
        testPosReviews.append(f.read())
for tfile in testNegFiles:
    with open(path+"test/neg/"+tfile, encoding="latin1") as f:
        testNegReviews.append(f.read())

In [5]:
reviews = pd.concat([
    pd.DataFrame({"review":positiveReviews, "label":1}),
    pd.DataFrame({"review":negativeReviews, "label":-1})
], ignore_index=True).sample(frac=1, random_state=1).reset_index(drop=True)

In [6]:
reviews.head()

,review,label
0,"I haven't seen ""Henry Fool"", but after watchin...",-1
1,"""The Groove Tube"" was one of only two Ken Shap...",1
2,My wife and I watch a film every night with no...,-1
3,"well, the writing was very sloppy, the directi...",-1
4,This movie captures the absurd essence of an o...,1


In [6]:
stopWords = stopwords.words('english')

In [7]:
def CleanData(sentence):
    processedList = ""
    
    #convert to lowercase and ignore special charcter
    sentence = re.sub(r'[^A-Za-z0-9\s.]', r'', str(sentence).lower())
    sentence = re.sub(r'\n', r' ', sentence)
    
    sentence = " ".join([word for word in sentence.split() if word not in stopWords])
    
    return sentence

In [9]:
reviews['processed_reviews'] = reviews['review'].map(lambda x: CleanData(x))

In [10]:
reviews.head()

,review,label,processed_reviews
0,"I haven't seen ""Henry Fool"", but after watchin...",-1,havent seen henry fool watching fay grim im su...
1,"""The Groove Tube"" was one of only two Ken Shap...",1,groove tube one two ken shapiro movies one equ...
2,My wife and I watch a film every night with no...,-1,wife watch film every night distractions mostl...
3,"well, the writing was very sloppy, the directi...",-1,well writing sloppy directing sloppier editing...
4,This movie captures the absurd essence of an o...,1,movie captures absurd essence overbearing amer...


In [11]:
def to_vw_format(document, label=None):
      return str(label) + ' |text ' + document

In [12]:
vw = pyvw.vw('--loss_function hinge')

for index, row in reviews.iterrows():
    vw_text = to_vw_format(row['processed_reviews'], 1 if row['label'] == 1 else -1)
    vw.learn(vw_text)

In [13]:
# save model
vw.save('imdb.model')